In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
def score(read_file_path,read_record_path,read_ground_path):
    record_max_score = []
    record_score = []
    record_file  =  np.genfromtxt((read_record_path+"/start_plant_list.csv"), delimiter=',', dtype=np.int32)
    record_fault = np.genfromtxt((read_record_path+"/start_fault_list.csv"), delimiter=',', dtype=np.int32)
    record_vali = np.genfromtxt((read_file_path+"/pred_start/record_vali.csv"), delimiter=',', dtype=np.int32)
    total_TP = 0
    total_FP = 0 
    total_FN = 0
    total_MS = 0
    
    #loop 
    for n_index in range(0,2):
        
        plant_ans_name = read_ground_path+"/PHM_testLabel_hidden"+str(record_file[n_index])+".csv"
        plant_ans = pd.read_csv(plant_ans_name)

        ##record_ground_ans
        ground_ans = plant_ans.loc[plant_ans.fault == record_fault[n_index],:]
        ground_ans.index = range( ground_ans.shape[0])
        print("here is: " +str(n_index)+" over: "+ str(len(record_file)) + " ans: "+str(ground_ans.shape[0]) + " plant: "+str(record_file[n_index]) + " fault: "+str(record_fault[n_index]))
        
        ##record start TP
        record_start_tp = ground_ans
        label = pd.DataFrame({"label":[0]*ground_ans.shape[0] })
        record_start_tp = pd.concat([record_start_tp,label], axis = 1,)
        ##record_other_ans
        other_ans = plant_ans.loc[plant_ans.fault != record_fault[n_index],:]
        other_ans.index = range( other_ans.shape[0])

        ##record_max_score
        record_max_score.append(ground_ans.shape[0]*10)
        if(record_vali[n_index] == 1):
            pred_ans_name = read_file_path + "/ans/ans_plant"+str(record_file[n_index])+"_fault"+str(record_fault[n_index])+".csv"
            pred_ans = pd.read_csv(pred_ans_name,header = 0,names = ["start_time","end_time"])
            
            
            ##calulate TP and MS
            S_TP = 0
            TP = 0
            MS = 0
            for i in range(pred_ans.shape[0]):
                ps = int(pred_ans.start_time[i])
                ps_series = range(ps-4,ps+5)
                filter_start = ground_ans.loc[ground_ans.startTime.isin(ps_series),]
                if filter_start.shape[0]>0:
                    record_start_tp.loc[record_start_tp.startTime.isin(filter_start.startTime),"label"] = 1
                    S_TP +=1
                    pe = int(pred_ans.end_time[i])
                    pe_series = range(pe-4,pe+5)
                    filter_end = filter_start.loc[filter_start.endTime.isin(pe_series),:]
                    if filter_end.shape[0] > 0:
                        ground_ans = ground_ans.loc[(ground_ans.index == filter_end.index[0]) == False,:]
                        TP+=1
                else:
                    filter_ms_start = other_ans.loc[other_ans.startTime.isin(ps_series),]
                    if filter_ms_start.shape[0]>0:
                        pe = int(pred_ans.end_time[i])
                        pe_series = range(pe-4,pe+5)
                        filter_ms_end = filter_ms_start.loc[filter_ms_start.endTime.isin(pe_series),:]
                        if filter_ms_end.shape[0] > 0:
                            other_ans = other_ans.loc[(other_ans.index == filter_ms_end.index[0]) == False,:]
                            MS+=1
            FP = pred_ans.shape[0] - TP - MS 
            FN = ground_ans.shape[0]
            score = TP*10-0.1*(FP+FN)-0.01*MS
            record_score.append(score)
            total_TP = total_TP + TP
            total_FP = total_FP + FP 
            total_FN = total_FN + FN
            total_MS = total_MS + MS
            print("S_TP: %s TP: %s Real_S_TP: %s FP: %s MS: %s socre: %s max: %s"
                  %(S_TP, TP, record_start_tp.label.sum(),FP , MS, score , record_max_score[-1]) )
        else:
            record_score.append(0)
        
    
    ##write csv
    d = { "get_score":record_score,"max_score":record_max_score}
    df = pd.DataFrame(d)
    write_name = read_file_path+"/fixold_score.csv"
    df.to_csv(write_name,index = False,sep = ",")
    
    confusion_list = [total_TP, total_FP, total_FN, total_MS]
    f_name = read_file_path+"/fixold_confusion_list.csv"
    f = open( f_name,"w")
    f.write(str(confusion_list[0]))
    for data in confusion_list[1:]:
        f.write("," + str(data))
    f.close()

In [3]:
read_file_path = "/media/joeytu/82F6C88FF6C8853F/PHM2017/20170831DEMO/file"
read_record_path = "/media/joeytu/82F6C88FF6C8853F/PHM2017/PHM_nosample/file"
read_ground_path = "/media/joeytu/82F6C88FF6C8853F/PHM2017/PHM_rawdata"

score(read_file_path,read_record_path,read_ground_path)

here is: 0 over: 135 ans: 185 plant: 1 fault: 1
S_TP: 208 TP: 110 Real_S_TP: 172 FP: 1914 MS: 37 socre: 900.73 max: 1850
here is: 1 over: 135 ans: 8 plant: 1 fault: 2
S_TP: 3 TP: 1 Real_S_TP: 3 FP: 39 MS: 0 socre: 5.3999999999999995 max: 80
